In [ ]:
# Requirements for this notebook to run: pandas, shapely, pyproj
import concurrent.futures
import time
import json

import pandas
from shapely.geometry import shape

from eocalc.context import Pollutant
from eocalc.methods.base import DateRange
from eocalc.methods.dummy import DummyEOEmissionCalculator
from eocalc.methods.fluky import RandomEOEmissionCalculator

# 1. Provide inputs

In [ ]:
with (open("data/regions/germany.geo.json", 'r')) as geojson_file:
    region = shape(json.load(geojson_file)["geometry"])
periods = [DateRange(start='2019-01-01', end='2019-12-31'),
          DateRange(start='2020-01-01', end='2020-12-31'),
          DateRange(start='2021-01-01', end='2021-12-31')]
methods = [DummyEOEmissionCalculator, RandomEOEmissionCalculator]
pollutants = [Pollutant.NOx, Pollutant.SO2, Pollutant.NH3]

In [ ]:
results = {} # results will be put here as results[<method>][<period index>][<pollutant>][<data>]

# 2. Check inputs
See wiki

# 3. Run methods

In [ ]:
for index, period in enumerate(periods):
    print(f"Processing period from {period.start} to {period.end} ({(period.end-period.start).days} days)")
    for method in methods:
        results.setdefault(method, {})
        for pollutant in pollutants:
            if method.supports(pollutant):
                print(f"Deriving {pollutant} emissions for given region using method {method.__name__}")
                results[method].setdefault(index, {})
                calculator = method()
                print(f"{calculator.state} at {calculator.progress}%")
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    future = executor.submit(calculator.run, region, period, pollutant)
                    while not future.done():
                        print(f"{calculator.state} at {calculator.progress}%")
                        time.sleep(1)
                    print(f"{calculator.state}")
                    results[method][index][pollutant] = future.result()

# 4. Display results

In [ ]:
results[DummyEOEmissionCalculator][0][Pollutant.NOx]

In [ ]:
results[RandomEOEmissionCalculator][1][Pollutant.SO2]['totals']

In [ ]:
results[RandomEOEmissionCalculator][1][Pollutant.SO2]['grid']

In [ ]:
results[RandomEOEmissionCalculator][1][Pollutant.SO2]['grid'].plot("SO2 [kt]", legend=True, figsize=(15, 15), alpha=0.6, edgecolor='k')